In [13]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.utils import negative_sampling
import torch
import numpy as np
import pandas as pd
import copy
from torch_geometric.utils.dropout import dropout_adj
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from utils.boolODE_data_to_pyg_data import make_adj_from_df, to_pyg_data
from sklearn.metrics import roc_auc_score
import math

In [14]:
# set user parameters:
#   datadir: directory in which data in the BoolODE format is available
#   name: name of the directory in which the data is located (subdirectory of datadir)
#   filenm: name under which the results should be saved for this network, note: output/"+filenm+"/"+filenm+"/" should exist before running!
#   num_features: amount of cells available for the data (2000 for mCAD example network)
datadir = 'data/'
name = 'mCAD/mCAD-2000-1'
filenm = 'mCAD_new'
num_features = 2000

# read out adjacency matrix and reference (true) network 
df=pd.read_csv(datadir + name + '/ExpressionData.csv', index_col=0)
adj_df = pd.read_csv(datadir + name + '/refNetwork.csv', index_col=0)
mat = df.to_numpy()

# construct Pytorch Geometric data object based on the input
sz = df.to_numpy().shape
edge_index, adj = make_adj_from_df(datadir,df, name)
true_data = to_pyg_data(mat, sz[0], sz[1], edge_index=edge_index)

ode_dim = true_data.x.shape[0]



In [15]:
# Define Graph Autoencoder (GAE) Model

class GAE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=16):
        super(GAE, self).__init__()

        # first graphSAGE layer
        self.conv1 = SAGEConv(input_dim, hidden_dim)

        # second graphSAGE layer
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)

        # dropout layer
        self.dropout = torch.nn.Dropout(0.3)

        # one linear layer (only weights) for decoding
        self.lin1 = torch.nn.Linear(hidden_dim, hidden_dim, bias=False)

    # encode node features
    def encode(self, data):

        # dropout edges to avoid overfitting
        edge_index = dropout_adj(data.edge_index, p = 0.2)[0]

        # apply first graphSAGE layer + activation function (ReLU)
        x = self.conv1(data.x, edge_index)
        x = F.relu(x)

        # apply dropout layer
        x = self.dropout(x)

        # apply second graphSAGE layer and return
        return self.conv2(x, edge_index)

    # decode specific edges
    def decode(self, z, edge_index):
        return (z[edge_index[0]] * self.lin1(z[edge_index[1]])).sum(dim=-1)  # Inner product
    
    # decode all edges for full adjacency matrix inference
    def decode_all(self,z):
        adj_matrix = torch.ones((z.shape[0], z.shape[0]))
        full_edge_index = adj_matrix.nonzero().t().contiguous()
        
        # inner product decoder with linear weights applied (lin1)
        return (z[full_edge_index[0]] * self.lin1(z[full_edge_index[1]])).sum(dim=-1)

In [16]:
# code for one training step
def train(model, data, query, optimizer, criterion):
    model.train()
    optimizer.zero_grad()

    # run data through encoder
    z = model.encode(data)

    # find appropriate negative examples of edges (not known edges, nor the edge that we want to predict)
    neg_edges = negative_sampling(torch.cat([data.edge_index,query],dim=1), data.num_nodes, data.edge_index.size(1))
    edges = torch.cat([data.edge_index, neg_edges], dim=1)
    
    # set labels: 1 for real edges, 0 for negative samples
    labels = torch.cat([torch.ones(data.edge_index.size(1)), torch.zeros(neg_edges.size(1))]).to(data.x.device)

    # decode for the true edges in the given GRN + the negative sampled edges
    preds = model.decode(z, edges)
    
    # optimization step
    loss = criterion(preds, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# code for training a model (multiple epochs)
def train_model(data, query, device):

    # initialise GAE model
    model = GAE(input_dim=num_features,hidden_dim=200)
    model.to(device)

    # set optimiser and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    criterion = torch.nn.BCEWithLogitsLoss()

    loss_vec = []

    # train for 1000 epochs
    for epoch in range(1000):
        loss = train(model, data, query, optimizer, criterion)
        
        loss_vec.append(loss)

    # return the trained GAE model
    return model

In [19]:
# first edge is removed from the true given GRN (as an example, in a practical use case this step can be skipped, as no edge needs to be removed)
rem = 0
imputed_edge_index = true_data.edge_index

mask = torch.ones(imputed_edge_index.shape[1], dtype=torch.bool)
mask[rem] = False

imputed_edge_index = imputed_edge_index[:,mask]

# create a new data object that has all gene expression data, and one fewer edge than the original data (the i-th edge)
data = copy.deepcopy(true_data)
data.edge_index = imputed_edge_index

# next, we find all edges that we want to predict, starting from a fully connected network
adj_matrix = torch.ones((ode_dim, ode_dim))

query_edge_index = adj_matrix.nonzero().t().contiguous()

# we remove all edges that are already in the given GRN from the edges to query
rem_query = []
for k in range(0,len(query_edge_index[0])):
    for j in range(0,len(data.edge_index[0])):
        if query_edge_index[0][k] == data.edge_index[0][j] and query_edge_index[1][k] == data.edge_index[1][j]:
            rem_query.append(k)

mask = torch.ones(query_edge_index.shape[1], dtype=torch.bool)
mask[rem_query] = False

query_edge_index = query_edge_index[:,mask]

# set device and move data to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = data.to(device)

# adj is the inferred adjacency matrix, whereas count_adj counts how many times an edge has been predicted (in case multiple runs are used)
adj = torch.zeros((ode_dim, ode_dim))
count_adj = torch.zeros((ode_dim, ode_dim))

# we train the GAE once for each query edge, resulting in a predicted adjacency matrix
for k in range(0,len(query_edge_index[0])):

    # set queried edge
    query = torch.tensor([[query_edge_index[0][k]],[query_edge_index[1][k]]])
    query = query.to(device)

    # train a new model to predict whether the queried edge is in the GRN
    model = train_model(data, query, device)
    model.eval()
    
    # use trained model to predict queried edge
    z = model.encode(data)
    dec = model.decode(z, query)

    # add each queried edge to the adjacency matrix
    for k in range(0, len(query[0])):
        adj[query[0][k], query[1][k]] = torch.sigmoid(dec[k])
        count_adj[query[0][k], query[1][k]] += 1

from scipy.io import savemat

# if edges are inferred multiple times, divide by the number of times they have reappeared
inferred_adj = (adj/count_adj).detach().numpy()

In [ ]:
# training procedure for benchmarking a known true network (looping over edges, removing then and then using GAE to predict), see above for more concise code to be used if one prediction is desired based on a given network
for i in range(0,len(true_data.edge_index[0])):

    # i-th edge is removed from the true given GRN
    rem = i
    imputed_edge_index = true_data.edge_index

    mask = torch.ones(imputed_edge_index.shape[1], dtype=torch.bool)
    mask[rem] = False

    imputed_edge_index = imputed_edge_index[:,mask]

    # create a new data object that has all gene expression data, and one fewer edge than the original data (the i-th edge)
    data = copy.deepcopy(true_data)
    data.edge_index = imputed_edge_index

    # next, we find all edges that we want to predict, starting from a fully connected network
    adj_matrix = torch.ones((ode_dim, ode_dim))

    query_edge_index = adj_matrix.nonzero().t().contiguous()

    # we remove all edges that are already in the given GRN from the edges to query
    rem_query = []
    for k in range(0,len(query_edge_index[0])):
        for j in range(0,len(data.edge_index[0])):
            if query_edge_index[0][k] == data.edge_index[0][j] and query_edge_index[1][k] == data.edge_index[1][j]:
                rem_query.append(k)

    mask = torch.ones(query_edge_index.shape[1], dtype=torch.bool)
    mask[rem_query] = False

    query_edge_index = query_edge_index[:,mask]

    # set device and move data to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    data = data.to(device)

    # adj is the inferred adjacency matrix, whereas count_adj counts how many times an edge has been predicted (in case multiple runs are used)
    adj = torch.zeros((ode_dim, ode_dim))
    count_adj = torch.zeros((ode_dim, ode_dim))

    # we train the GAE once for each query edge, resulting in a predicted adjacency matrix
    print("Training for edge "+str(i+1)+"/"+str(len(true_data.edge_index[0])))

    for k in range(0,len(query_edge_index[0])):

        # set queried edge
        query = torch.tensor([[query_edge_index[0][k]],[query_edge_index[1][k]]])
        query = query.to(device)

        # train a new model to predict whether the queried edge is in the GRN
        model = train_model(data, query, device)
        model.eval()
        
        # use trained model to predict queried edge
        z = model.encode(data)
        dec = model.decode(z, query)

        # add each queried edge to the adjacency matrix
        for k in range(0, len(query[0])):
            adj[query[0][k], query[1][k]] = torch.sigmoid(dec[k])
            count_adj[query[0][k], query[1][k]] += 1

    from scipy.io import savemat

    # if edges are inferred multiple times, divide by the number of times they have reappeared
    inferred_adj = (adj/count_adj).detach().numpy()

    # save inferred adjacency to a .mat file
    i1 = true_data.edge_index[:,i][0].numpy()
    i2 = true_data.edge_index[:,i][1].numpy()
    savemat("output/"+filenm+"/"+filenm+"_"+str(i1)+"_"+str(i2)+".mat",{"inferred_adj": inferred_adj})

    inferred_adj = (adj/count_adj).detach().numpy()
    N = inferred_adj.shape[0]

    # Ground truth positive edges
    pos_edges = set((i.item(), j.item()) for i, j in edge_index.t())

    # All possible (i, j) pairs excluding self-loops
    all_edges = [(i, j) for i in range(N) for j in range(N)]

    # optional computation of AUROC for predicted edge
    y_true = []
    y_score = []

    for i, j in all_edges:
        score = inferred_adj[i, j].item()
        # skip all NaN values (edges that we already knew)
        if math.isnan(score):
            continue
        y_true.append(1 if (i, j) in pos_edges else 0)
        y_score.append(score)

    auroc = roc_auc_score(y_true, y_score)

    

Training for edge 1/14


C:\Users\JansenKlompLF\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch_geometric\deprecation.py:21: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


KeyboardInterrupt: 